In [11]:
7

7

In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 12.6/12.6 MB 1.2 MB/s eta 0:00:00
     ------------------------------------ 138.3/138.3 kB 547.2 kB/s eta 0:00:00
     -------------------------------------- 96.9/96.9 kB 692.9 kB/s eta 0:00:00
     ------------------------------------ 189.2/189.2 kB 602.5 kB/s eta 0:00:00
     ------------------------------------ 229.1/229.1 kB 500.9 kB/s eta 0:00:00
     -------------------------------------- 48.8/48.8 kB 189.8 kB/s eta 0:00:00
     ------------------------------------ 413.4/413.4 kB 806.2 kB/s eta 0:00:00
     ------------------------------------ 151.7/151.7 kB 822.4 kB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [21]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os
from googleapiclient.http import MediaIoBaseDownload
import io
import pandas as pd

# Scopes required by the app

def get_service():
    SCOPES = ['https://www.googleapis.com/auth/drive.file']

    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service


def gdrive_file_upload(filename, gdrive_folder_id):
    service = get_service()

    # File to upload
    file_metadata = {'name': filename, 'parents': [gdrive_folder_id]}
    media = MediaFileUpload(filename, mimetype='application/zip')
    file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()
    file_id = file.get('id')
    csv_file = 'log.csv'

    df = pd.DataFrame(columns=['File Name', 'File ID', 'time'])
    df = df.append({'File Name': filename, 'File ID': file_id, 'time': pd.Timestamp.now()}, ignore_index=True)
    if os.path.exists(csv_file):
        df.to_csv(csv_file, mode='a', header=False, index=False)
    
    return file.get('id')

def gdrive_file_download(filename, gdrive_file_id):
    service = get_service()
    request = service.files().get_media(fileId=gdrive_file_id)
    
    # Replace 'filename' with the path where you want to save the downloaded file
    fh = io.FileIO(filename, mode='wb')
    downloader = MediaIoBaseDownload(fh, request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print("Download Progress: {0}".format(status.progress() * 100))

    fh.close()
    print("File has been downloaded successfully.")



In [22]:
file = 'data/example1.zip'
gdrive_folder_id = '1te7ArAHuMquqmWhjmO-FniTE47O4mts7'
uploaded_file_id = gdrive_file_upload(file, gdrive_folder_id)
print(f"File has been uploaded successfully. File ID: {uploaded_file_id}")

AttributeError: 'DataFrame' object has no attribute 'concat'

In [10]:
file = 'data/filename.zip'
gdrive_file_id = uploaded_file_id
gdrive_file_download(file, gdrive_file_id)

Download Progress: 100.0
File has been downloaded successfully.
